LSE Data Science Institute | ME204 (2023/24) | Week 03 Day 01

# 🗓️ Week 03 – Day 01: Intro to Databases and SQL


<span style="display: inline-block; padding: 0 10px; font-size: 1.15em;line-height: 1.5em; white-space: nowrap; border: 1px solid #3995ba; border-radius: .5em; color: #fcfcfc; background-color: #3995ba; vertical-align: middle;font-weight: 600 !important;">MORNING NOTEBOOK</span>

**DATE:** 22 July 2024

**AUTHOR:** Alex Soldatkin & Dr [Jon Cardoso-Silva](https://jonjoncardoso.github.io)

-----


# ⚙️ Setup

You can use your `base` environment or create a new one for this notebook. 

These are the libraries you need to install:

<div style="width:60%;font-size:0.85em">

```bash
pip install pandas numpy lets-plot numerize tqdm sqlalchemy jupysql
```

💡 **NOTE**: There are a few new ones today: the [`jupysql`](https://jupysql.ploomber.io/en/latest/intro.html) and [`sqlalchemy`](https://www.sqlalchemy.org/) packages.

</div>

**IMPORTS:**

In [1]:
import os
import sqlite3


import numpy as np
import pandas as pd

from tqdm.notebook import tqdm, trange
tqdm.pandas()

**CONSTANTS:**

In [2]:
# Adjust it according to your data folder
DATA_FOLDER = os.path.join('../data/')

WAITROSE_FOLDER = os.path.join(DATA_FOLDER, 'waitrose')

In [3]:
%load_ext sql
%config SqlMagic.autocommit=True # for engines that do not support autommit

**READ THE RAW DATA**

Read the bunch of CSV files we got from scraping the Waitrose website.

In [233]:
# Run this very long and ugly cell to get the full dataset
all_files = [os.path.join(WAITROSE_FOLDER,  file) for file in os.listdir(WAITROSE_FOLDER) 
             if file.endswith('.csv')]

# Reads every individual file and concatenates them into a single DataFrame using pandas concat
df = pd.concat((pd.read_csv(file) for file in all_files))

df = df.drop_duplicates()

df = df.drop(columns=['data-product-name'])
df = (
    df.rename(columns={
        'data-product-type': 'type',
        'data-product-index': 'index',
        'data-product-id': 'id',
        'data-product-on-offer': 'on-offer',
        'product-page': 'page',
        'product-name': 'name',
        'product-size': 'size',
    })
)

def clean_item_price(item_price: str):
    """
    Cleans the item price string by performing necessary transformations.

    Parameters:
    item_price (str): The item price as a string.

    Returns:
    float: The cleaned item price.
    """
    if pd.isna(item_price):
        return np.nan
    
    item_price = item_price.replace(' each est.', '')
    
   
    if '-' in item_price:
        item_prices = item_price.replace('£', '').split('-')
        item_prices = [float(price) for price in item_prices]
        item_price = np.mean(item_prices)
    else:
        if '£' in item_price:
            item_price = item_price.replace('£', '')
            item_price = float(item_price)
        elif 'p' in item_price:
            item_price = item_price.replace('p', '')
            item_price = float('0.' + item_price)
    
    return float(item_price)

df['id'] = df['id'].astype('int32')
df['cleaned-item-price'] = df['item-price'].apply(lambda price: clean_item_price(price))

selected_rows = df['item-price'].astype(str).str.contains('-')

# The 'size' column of the selected rows all have the same kind of format: (2.2kg-2.8kg).
df.loc[selected_rows, 'size'] = df.loc[selected_rows, 'size'].str.split('-').str[0].str.replace('(', '')

df['item-price'] = df['cleaned-item-price']
df.drop(columns=['cleaned-item-price', 'index'], inplace=True)

df

,id,type,on-offer,image-url,page,name,size,item-price,price-per-unit,offer-description,category
0,497633,G,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/finish-...,Finish Quantum All in One Lemon Dishwasher Tab...,64Each,10.00,15.6p each,Save 1/3. Was £15.00,Household
1,627645,G,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/finish-...,Finish Ultimate Plus All in One Lemon Dishwash...,48Each,9.00,18.8p each,save £6.50. Was £15.50,Household
2,475915,G,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/finish-...,Finish Ultimate All in One Lemon Dishwasher Ta...,36s,8.00,22.2p each,Save 1/3. Was £12.50,Household
3,28863,G,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential White Ultra Soft Bathroom Tissue,9x190 sheets,4.00,23.4p/100 sheets,NaN,Household
4,22482,G,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/essenti...,Essential Thick Bleach Citrus,750ml,0.75,£1/litre,NaN,Household
...,...,...,...,...,...,...,...,...,...,...,...
1290,16521,G,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/watergu...,Watergull Orchards Apple Juice Cox,750ml,2.65,35.3p/100ml,NaN,"Tea, Coffee & Soft Drinks"
1291,16118,G,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/red-bul...,Red Bull Energy Drink,4x250ml,4.00,40p/100ml,save £1.50. Was £5.50,"Tea, Coffee & Soft Drinks"
1292,12467,G,True,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/red-bul...,Red Bull Sugarfree,4x250ml,4.00,40p/100ml,save £1.25. Was £5.25,"Tea, Coffee & Soft Drinks"
1293,7932,G,False,https://ecom-su-static-prod.wtrecom.com/images...,https://www.waitrose.com/ecom/products/pixley-...,Pixley Berries Blackcurrant Cordial,500ml,4.75,95p/100ml,NaN,"Tea, Coffee & Soft Drinks"


**CONNECT TO THE DATABASE**

In [5]:
conn = sqlite3.connect(os.path.join(DATA_FOLDER, 'supermarket.db'))

## SOLUTION 01: NO FORMAL KEY CONSTRAINTS (easyer to implement but less ideal)


<div style="width:400px;height:260px;border-radius:1em;margin:0.5%;padding:1%;background-color:#fafafa">

<h2>💡 Read this!</h2>

In this solution, we will create multiple tables to help us exercise linking them together with `pandas` later. I will not use primary or foreign keys. Instead, I will derive the separate data frames from the main one, making sure that there are no duplicates left, before I dump them into the database.

I believe this might be easier for those who are just starting with SQL, with the caveat that <span style="color:red"> this is not the best practice for databases</span>. Ideally, you want to enforce constraints (primary and foreign keys) to ensure data integrity -- and we can't get that with this approach.
</div>


### Table 01: `products`

<details style="color:transparent;background-color:transparent;height:0px">

It turns out that IDs are truly unique, we do not need (id, name, size) like we've been doing in the past. This saves us a lot of time and effort. We can just use the `id` as the primary key for the `products` table.

To confirm that:

```python
(
    df
     .groupby(['id'])
     .apply(lambda x: pd.Series({'num_unique_pages': x['page'].nunique()}))
     .sort_values('num_unique_pages', ascending=False)
     .query('num_unique_pages > 1')
)
````

</details>


We know that we **want** to exercise the linking of tables, so we will not include the `category` column in this table. We will create a separate table for that. That also applies for the information about prices and offers.

This leaves us with the following columns for the `products` table:

- `id` (int): the unique identifier for the product (as determined by Waitrose)
- `name` (str): the name of the product
- `size` (str): the size of the product
- `page` (str): the URL of the product's page on the Waitrose website
- `image-url` (str): the URL of the product's image
- `type` (str): the type of the product (which, to be honest, I don't know what it means)

👉 **These are all the attributes that are specific to the product itself and not to its price or offer or category.**

Let's build the dataframe for the `products` table.

In [258]:
selected_columns = ['id', 'name', 'size', 'page', 'image-url', 'type']

products = df[selected_columns].copy().drop_duplicates()

print(f"While the original dataset has {len(df)} rows, the cleaned dataset has {len(products)} rows, which means that {len(df) - len(products)} rows were removed.")

While the original dataset has 25378 rows, the cleaned dataset has 16065 rows, which means that 9313 rows were removed.


What is in the `products` dataframe?

In [131]:
products

,id,name,size,page,image-url,type
0,497633,Finish Quantum All in One Lemon Dishwasher Tab...,64Each,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G
1,627645,Finish Ultimate Plus All in One Lemon Dishwash...,48Each,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G
2,475915,Finish Ultimate All in One Lemon Dishwasher Ta...,36s,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G
3,28863,Essential White Ultra Soft Bathroom Tissue,9x190 sheets,https://www.waitrose.com/ecom/products/essenti...,https://ecom-su-static-prod.wtrecom.com/images...,G
4,22482,Essential Thick Bleach Citrus,750ml,https://www.waitrose.com/ecom/products/essenti...,https://ecom-su-static-prod.wtrecom.com/images...,G
...,...,...,...,...,...,...
1285,29679,Jacksons Fairtrade Sencha Green Tea Bags 20,50g,https://www.waitrose.com/ecom/products/jackson...,https://ecom-su-static-prod.wtrecom.com/images...,G
1290,16521,Watergull Orchards Apple Juice Cox,750ml,https://www.waitrose.com/ecom/products/watergu...,https://ecom-su-static-prod.wtrecom.com/images...,G
1291,16118,Red Bull Energy Drink,4x250ml,https://www.waitrose.com/ecom/products/red-bul...,https://ecom-su-static-prod.wtrecom.com/images...,G
1292,12467,Red Bull Sugarfree,4x250ml,https://www.waitrose.com/ecom/products/red-bul...,https://ecom-su-static-prod.wtrecom.com/images...,G


Let's save it to the database.

In [138]:
products.to_sql('products', conn, if_exists='replace', index=False)

16065

Test that it worked:

In [139]:
pd.read_sql('SELECT COUNT(*) AS total_num_products FROM products', conn)

,total_num_products
0,16065


In [140]:
# See what the table looks like, get just the first 5 rows
pd.read_sql('SELECT * FROM products LIMIT 5', conn)

,id,name,size,page,image-url,type
0,497633,Finish Quantum All in One Lemon Dishwasher Tab...,64Each,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G
1,627645,Finish Ultimate Plus All in One Lemon Dishwash...,48Each,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G
2,475915,Finish Ultimate All in One Lemon Dishwasher Ta...,36s,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G
3,28863,Essential White Ultra Soft Bathroom Tissue,9x190 sheets,https://www.waitrose.com/ecom/products/essenti...,https://ecom-su-static-prod.wtrecom.com/images...,G
4,22482,Essential Thick Bleach Citrus,750ml,https://www.waitrose.com/ecom/products/essenti...,https://ecom-su-static-prod.wtrecom.com/images...,G


### Table 2: `categories` (DIFFICULT BUT GOOD LEARNING OPPORTUNITY)

Previously, I showed you an example where we created a very simple `categories` table with only two columns: `id` and `name`. This is common practice in databases when you need to keep track of additional information about the categories. Say, for example, you want to keep track of the number of products in each category, when they started being sold, when they became inactive, etc. In that scenario, I would ideally create a third table, just to map `category_id` to `product_id`. This is also called a [**many-to-many relationship**](https://www.geeksforgeeks.org/relationships-in-sql-one-to-one-one-to-many-many-to-many/).

Since we don't have additional info on the categories themselves, I will simplify things and create just one table that relates to `categories`. In this table, each product can be associated with multiple categories. This is a [**one-to-many relationship**](https://www.geeksforgeeks.org/relationships-in-sql-one-to-one-one-to-many-many-to-many/).

This table will have the following columns:

- `product_id` (int): the unique identifier for the product
- `category_name` (int): the name of the category

That's it! By moving it to a separate table, we keep the `products` table free of duplicates and make it easier to update the categories for a product.

Let's write the code to create this table:

In [152]:
# This is how the table looks like
categories = (
    df[['id', 'category']].drop_duplicates()
    .rename(columns={'category': 'name', 'id': 'product_id'})
)

Time to save it:

In [153]:
categories.to_sql('categories', conn, if_exists='replace', index=False)

25377

**Test that it works**

In [154]:
pd.read_sql('SELECT * FROM categories LIMIT 5', conn)

,product_id,name
0,497633,Household
1,627645,Household
2,475915,Household
3,28863,Household
4,22482,Household


#### What if I want to get the categories of a product under this new schema?

In [157]:
# We select the products and join them with the categories table to get the category names
pd.read_sql(
'''
SELECT
    p.*,
    c.name AS category
FROM
    products p
LEFT JOIN
    categories c
ON
    p.id = c.product_id
LIMIT 5
''', conn)
 

,id,name,size,page,image-url,type,category
0,497633,Finish Quantum All in One Lemon Dishwasher Tab...,64Each,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G,Household
1,627645,Finish Ultimate Plus All in One Lemon Dishwash...,48Each,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G,Household
2,475915,Finish Ultimate All in One Lemon Dishwasher Ta...,36s,https://www.waitrose.com/ecom/products/finish-...,https://ecom-su-static-prod.wtrecom.com/images...,G,Household
3,28863,Essential White Ultra Soft Bathroom Tissue,9x190 sheets,https://www.waitrose.com/ecom/products/essenti...,https://ecom-su-static-prod.wtrecom.com/images...,G,Everyday Value
4,28863,Essential White Ultra Soft Bathroom Tissue,9x190 sheets,https://www.waitrose.com/ecom/products/essenti...,https://ecom-su-static-prod.wtrecom.com/images...,G,Household


☝️ Notice how the products appear multiple times if they have more than one category. This is the expected behavior, as we are linking the two tables together with a simple JOIN operation.

**Here's an alternative way that preserves the unique products:**

In [158]:
# We group by the `product_id` and use GROUP_CONCAT to concatenate all the category names into a single string
pd.read_sql(
'''
SELECT
    p.*,
    GROUP_CONCAT(c.name) AS categories
FROM
    products p
LEFT JOIN
    categories c
ON
    p.id = c.product_id
GROUP BY
    p.id
LIMIT 5
''', conn)
 

,id,name,size,page,image-url,type,categories
0,1023,Taylors of Harrogate Yorkshire Gold 160 Bags,500g,https://www.waitrose.com/ecom/products/taylors...,https://ecom-su-static-prod.wtrecom.com/images...,G,"Tea, Coffee & Soft Drinks"
1,1029,Nestlé Box Bowls Variety 6 Portion Pack,210g,https://www.waitrose.com/ecom/products/nestle-...,https://ecom-su-static-prod.wtrecom.com/images...,G,Food Cupboard
2,1048,Kellogg's Corn Flakes Breakfast Cereal Large Pack,670g,https://www.waitrose.com/ecom/products/kellogg...,https://ecom-su-static-prod.wtrecom.com/images...,G,"Dietary & Lifestyle,Food Cupboard"
3,1051,Kellogg's Crunchy Nut Breakfast Cereal Large Pack,840g,https://www.waitrose.com/ecom/products/kellogg...,https://ecom-su-static-prod.wtrecom.com/images...,G,"Dietary & Lifestyle,Food Cupboard"
4,1052,Kellogg's Crunchy Nut Breakfast Cereal,460g,https://www.waitrose.com/ecom/products/kellogg...,https://ecom-su-static-prod.wtrecom.com/images...,G,"Dietary & Lifestyle,Food Cupboard"


### Table 3: `prices`

The `prices` table will have the following columns:

- `product_id` (int): the unique identifier for the product
- `item-price` (float): the price of the product
- `price-per-unit` (float): the price per unit of the product

The process is very similar to what we did for the `categories` table:

In [174]:
prices = (
    df[['id', 'item-price', 'price-per-unit']].drop_duplicates()
    .sort_values('id')
    .rename(columns={'id': 'product_id'})
)
prices.head()

,product_id,item-price,price-per-unit
314,1023,7.10,£1.42/100g
2403,1029,2.30,£1.10/100g
513,1048,2.75,41p/100g
794,1051,5.00,59.5p/100g
805,1052,2.75,59.8p/100g


Save it:

In [175]:
prices.to_sql('prices', conn, if_exists='replace', index=False)

16065

Test that the table was created correctly

In [176]:
pd.read_sql('SELECT * FROM prices LIMIT 5', conn)

,product_id,item-price,price-per-unit
0,1023,7.10,£1.42/100g
1,1029,2.30,£1.10/100g
2,1048,2.75,41p/100g
3,1051,5.00,59.5p/100g
4,1052,2.75,59.8p/100g


For each product in the `products` table, recover their prices and categories.

In [178]:
pd.read_sql(
'''
SELECT
    p.*,
    pr.*,
    GROUP_CONCAT(DISTINCT c.name) AS categories
FROM
    products p
LEFT JOIN
    prices pr
ON
    p.id = pr.product_id
LEFT JOIN
    categories c
ON
    p.id = c.product_id
GROUP BY p.id
LIMIT 5
''', conn)


,id,name,size,page,image-url,type,product_id,item-price,price-per-unit,categories
0,1023,Taylors of Harrogate Yorkshire Gold 160 Bags,500g,https://www.waitrose.com/ecom/products/taylors...,https://ecom-su-static-prod.wtrecom.com/images...,G,1023,7.10,£1.42/100g,"Tea, Coffee & Soft Drinks"
1,1029,Nestlé Box Bowls Variety 6 Portion Pack,210g,https://www.waitrose.com/ecom/products/nestle-...,https://ecom-su-static-prod.wtrecom.com/images...,G,1029,2.30,£1.10/100g,Food Cupboard
2,1048,Kellogg's Corn Flakes Breakfast Cereal Large Pack,670g,https://www.waitrose.com/ecom/products/kellogg...,https://ecom-su-static-prod.wtrecom.com/images...,G,1048,2.75,41p/100g,"Dietary & Lifestyle,Food Cupboard"
3,1051,Kellogg's Crunchy Nut Breakfast Cereal Large Pack,840g,https://www.waitrose.com/ecom/products/kellogg...,https://ecom-su-static-prod.wtrecom.com/images...,G,1051,5.00,59.5p/100g,"Dietary & Lifestyle,Food Cupboard"
4,1052,Kellogg's Crunchy Nut Breakfast Cereal,460g,https://www.waitrose.com/ecom/products/kellogg...,https://ecom-su-static-prod.wtrecom.com/images...,G,1052,2.75,59.8p/100g,"Dietary & Lifestyle,Food Cupboard"


### Table 4: `offers`

The `offers` table will have the following columns:

- `product_id` (int): the unique identifier for the product 
- `offer-description` (str): the offer on the product


**⚠️ IMPORTANT**: I don't need to store a `is_on_offer` column in the `products` table. This is redundant information, as I can get this information from the `offers` table. When joining the two tables, I can check if the `description` column is not null.

In [187]:
offers = (
    df[df['on-offer'] == True][['id', 'offer-description']].drop_duplicates()
    .sort_values('id')
    .rename(columns={'id': 'product_id'})
)
offers

,product_id,offer-description
513,1048,save 45p. Was £3.20
805,1052,save 75p. Was £3.50
863,1066,save 70p. Was £3.45
1316,1138,save £1.30. Was £3.80
1248,1182,save 85p. Was £1.85
...,...,...
171,988412,Introductory Offer.Will be £6.00
265,990127,Introductory Offer.Will be £3.65
257,995459,New Line
249,999031,Introductory Offer.Will be £1.20


By filtering to only the products that are on offer, I get fewer rows in the `offers` table, which is good for performance.

In [188]:
offers.to_sql('offers', conn, if_exists='replace', index=False)

4656

To select **only the products that are on offer**, use the `INNER JOIN` operation:

In [192]:
pd.read_sql(
'''
SELECT
    p.id,
    p.name,
    o.`offer-description`
FROM
    products p
INNER JOIN
    offers o
ON 
    p.id = o.product_id
''', conn
)

,id,name,offer-description
0,497633,Finish Quantum All in One Lemon Dishwasher Tab...,Save 1/3. Was £15.00
1,627645,Finish Ultimate Plus All in One Lemon Dishwash...,save £6.50. Was £15.50
2,475915,Finish Ultimate All in One Lemon Dishwasher Ta...,Save 1/3. Was £12.50
3,14407,Andrex Supreme Quilts Toilet Roll,Add 2 for £10
4,71469,Andrex Classic Clean Toilet Roll,save £1.65. Was £6.60
...,...,...,...
4654,99326,Complan Original,Add 2 for £10
4655,82540,Fentimans Dandelion & Burdock,save 65p. Was £1.65
4656,54537,Nescafe Dolce Gusto Lungo Coffee Pods 16's,Add 2 for £7.50
4657,16118,Red Bull Energy Drink,save £1.50. Was £5.50


To add a `is-on-offer` column to the `products` table, use the `LEFT JOIN` operation and the `CASE` statement:

In [194]:
# Add is-on-offer column to the products table if there's no offer for a product, set it to False
pd.read_sql(
'''
SELECT
    p.id,
    p.name,
    o.`offer-description`,
    CASE
        WHEN o.`offer-description` IS NULL THEN False
        ELSE True
    END AS `is-on-offer`
FROM
    products p
LEFT JOIN
    offers o
ON
    p.id = o.product_id
''', conn
)

    

,id,name,offer-description,is-on-offer
0,497633,Finish Quantum All in One Lemon Dishwasher Tab...,Save 1/3. Was £15.00,1
1,627645,Finish Ultimate Plus All in One Lemon Dishwash...,save £6.50. Was £15.50,1
2,475915,Finish Ultimate All in One Lemon Dishwasher Ta...,Save 1/3. Was £12.50,1
3,28863,Essential White Ultra Soft Bathroom Tissue,None,0
4,22482,Essential Thick Bleach Citrus,None,0
...,...,...,...,...
16060,29679,Jacksons Fairtrade Sencha Green Tea Bags 20,None,0
16061,16521,Watergull Orchards Apple Juice Cox,None,0
16062,16118,Red Bull Energy Drink,save £1.50. Was £5.50,1
16063,12467,Red Bull Sugarfree,save £1.25. Was £5.25,1


---



In [198]:
# Check that all the tables are created correctly

pd.read_sql('SELECT * FROM sqlite_master', conn).set_index('sql')

,type,name,tbl_name,rootpage
sql,,,,
"CREATE TABLE ""products"" (\n""id"" INTEGER,\n ""name"" TEXT,\n ""size"" TEXT,\n ""page"" TEXT,\n ""image-url"" TEXT,\n ""type"" TEXT\n)",table,products,products,3
"CREATE TABLE ""categories"" (\n""product_id"" INTEGER,\n ""name"" TEXT\n)",table,categories,categories,2
"CREATE TABLE ""prices"" (\n""product_id"" INTEGER,\n ""item-price"" REAL,\n ""price-per-unit"" TEXT\n)",table,prices,prices,10
"CREATE TABLE ""offers"" (\n""product_id"" INTEGER,\n ""offer-description"" TEXT\n)",table,offers,offers,1310


## SOLUTION 2: MORE FORMAL KEY CONSTRAINTS (more complex but ideal)

The way to create the data frames that will eventually populate the database is the same as before. The difference is that we will now create the database schema with primary and foreign keys.

Therefore, we cannot use the `if_exists='replace'` parameter in the `to_sql` method. Otherwise, we will just be creating new tables every time we run the code.

In [200]:
# For this I'd rather use the jupysql package
%sql sqlite:///../data/supermarket.db --alias supermarket

Connecting to 'sqlite:///supermarket.db'

In [221]:
%%sql

-- RECREATE THE products TABLE

DROP TABLE IF EXISTS products;

CREATE TABLE "products" (
 "id" INTEGER PRIMARY KEY,
 "name" VARCHAR(100),
 "size" VARCHAR(30),
 "page" VARCHAR(300),
 "image-url" VARCHAR(300),
 "type" CHAR(1)
);

-- RECREATE THE categories TABLE

DROP TABLE IF EXISTS categories;

CREATE TABLE "categories" (
 "product_id" INTEGER,
 "name" VARCHAR(50),
 PRIMARY KEY ("product_id", "name"),
 FOREIGN KEY ("product_id") REFERENCES "products" ("id")
);

-- RECREATE THE prices TABLE

DROP TABLE IF EXISTS prices;

CREATE TABLE "prices" (
 "product_id" INTEGER,
 "item-price" REAL,
 "price-per-unit" TEXT,
  FOREIGN KEY ("product_id") REFERENCES "products" ("id")
);

-- RECREATE THE offers TABLE

DROP TABLE IF EXISTS offers;

CREATE TABLE "offers" (
  "product_id" INTEGER,
  "offer-description" TEXT,
  FOREIGN KEY ("product_id") REFERENCES "products" ("id")
)


Running query in 'sqlite:///supermarket.db'

++
||
++
++

**Populate the database**

When we try to populate the database, we get an error saying that some ids are duplicated!!

This is, of course, annoying because we now have to fix the problem but good because otherwise, we would have inserted the data into the database with duplicate ids.

In [222]:
products.to_sql('products', conn, if_exists='append', index=False)

IntegrityError: UNIQUE constraint failed: products.id

In [253]:
# Investigate it

# What are the duplicated ids?
duplicated_ids = products['id'][products['id'].duplicated()].tolist()

# Get the products with the duplicated ids
products[products['id'].isin(duplicated_ids)].sort_values('id')[['id', 'name', 'size']]

,id,name,size
48,67459,NaN,NaN
236,67459,No.1 Royal Deeside Still Water,750ml
340,69460,Waitrose Pineapple Fruit Juice,1litre
49,69460,NaN,NaN
48,85201,NaN,NaN
35,85201,Essential Leeks,Typical weight 0.21kg
34,85247,Essential Cauliflower,Each
48,85247,NaN,NaN
36,85513,Duchy Organic Carrots,650g
49,85513,NaN,NaN


This error is somewhat easy to fix! The duplicated entries are essentially **scraping errors**. We can just drop those that contain NaN values in the `name` column.

In [259]:
# Drop the rows with NaN values in the `name` column
products = products.dropna(subset=['name'])

Try again:

In [260]:
products.to_sql('products', conn, if_exists='append', index=False)

16054

IT WORKED! 

**Add the categories table**

In [261]:
categories.to_sql('categories', conn, if_exists='append', index=False)

25377

**Add the `prices` table to the database**

In [263]:
prices.to_sql('prices', conn, if_exists='append', index=False)

16065

**Add the `offers` table to the database**

In [264]:
offers.to_sql('offers', conn, if_exists='append', index=False)

4656

✅ AND WE'RE DONE! WE MADE SURE THAT EVERYTHING IS IN THE DATABASE AND THAT THE DATA IS CONSISTENT.